In [1]:
from torchvision.models import ResNet50_Weights

import src.utils as utils
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [2]:
df = utils.build_dataset('data/plantvillage/plantvillage dataset')
df

,Format,Species,Healthy,Disease,Folder,FileName
0,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,8f558908-aa1b-4a86-855a-5094c2392e5a___RS_HL 1...
1,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,b8e9ed27-8e37-4214-9206-f8c0ef21cf4d___RS_HL 4...
2,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,abdd34a0-ab02-41e0-95a3-a014ab863ec2___RS_HL 1...
3,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,d1aee44a-b6bb-45b9-b7b6-5d553add8fd1___RS_HL 2...
4,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,3d28c3ea-8419-4e09-addd-211e3828e39f___RS_HL 1...
...,...,...,...,...,...,...
162911,segmented,Soybean,True,None,data/plantvillage/plantvillage dataset/segment...,f6579a78-e6eb-4a65-82f7-7be30f100a07___RS_HL 5...
162912,segmented,Soybean,True,None,data/plantvillage/plantvillage dataset/segment...,356eb227-3e6d-4164-b84d-31f590293644___RS_HL 4...
162913,segmented,Soybean,True,None,data/plantvillage/plantvillage dataset/segment...,5d3def53-fdb2-4106-ad31-c020e75bccea___RS_HL 7...
162914,segmented,Soybean,True,None,data/plantvillage/plantvillage dataset/segment...,63d474df-5512-4ecc-9cd3-c0649c260668___RS_HL 7...


In [3]:
class PlantDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.df.iloc[idx]['Folder'], self.df.iloc[idx]['FileName'])
        image = Image.open(img_path).convert('RGB')
        label = 1 if self.df.iloc[idx]['Healthy'] else 0  # binary classification

        if self.transform:
            image = self.transform(image)
        return image, label


In [4]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [5]:
df = df[df['Healthy'].notna()]  # eliminamos filas sin etiqueta

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Healthy'], random_state=42)

train_dataset = PlantDataset(train_df, transform=train_transform)
val_dataset = PlantDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [6]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [7]:
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Reemplazamos la última capa para clasificación binaria
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

model = model.to(device)

# Congelamos todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Activamos solo los parámetros de la última capa (la nueva capa fc)
for param in model.fc.parameters():
    param.requires_grad = True

for name, param in model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")


conv1.weight: requires_grad = False
bn1.weight: requires_grad = False
bn1.bias: requires_grad = False
layer1.0.conv1.weight: requires_grad = False
layer1.0.bn1.weight: requires_grad = False
layer1.0.bn1.bias: requires_grad = False
layer1.0.conv2.weight: requires_grad = False
layer1.0.bn2.weight: requires_grad = False
layer1.0.bn2.bias: requires_grad = False
layer1.0.conv3.weight: requires_grad = False
layer1.0.bn3.weight: requires_grad = False
layer1.0.bn3.bias: requires_grad = False
layer1.0.downsample.0.weight: requires_grad = False
layer1.0.downsample.1.weight: requires_grad = False
layer1.0.downsample.1.bias: requires_grad = False
layer1.1.conv1.weight: requires_grad = False
layer1.1.bn1.weight: requires_grad = False
layer1.1.bn1.bias: requires_grad = False
layer1.1.conv2.weight: requires_grad = False
layer1.1.bn2.weight: requires_grad = False
layer1.1.bn2.bias: requires_grad = False
layer1.1.conv3.weight: requires_grad = False
layer1.1.bn3.weight: requires_grad = False
layer1.1.bn

In [8]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

model = model.to(device)  # asegurate de mover el modelo a GPU

for epoch in range(5):
    model.train()
    running_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)
    for inputs, labels in progress_bar:
        inputs = inputs.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        avg_loss = running_loss / (progress_bar.n + 1)

        progress_bar.set_postfix(loss=avg_loss)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import ResNet50_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm

# === Configuración
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
lr = 1e-4
epochs = 10

print(device)

# === Cargar ResNet50 preentrenado
base_model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Congelar todas las capas
for param in base_model.parameters():
    param.requires_grad = False

# === Redefinir la cabeza del modelo
class CustomResNet50(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.backbone = nn.Sequential(*list(base_model.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.head = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(base_model.fc.in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)  # Sin sigmoid
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.head(x)
        return x

model = CustomResNet50(base_model).to(device)

# Mostrar qué parámetros están siendo entrenados
for name, param in model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

# === Optimización y pérdida
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
criterion = nn.BCEWithLogitsLoss()

# === Entrenamiento
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0

    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)
    for inputs, labels in train_bar:
        inputs = inputs.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        avg_loss = running_loss / (train_bar.n + 1)
        acc = correct / total
        train_bar.set_postfix(loss=avg_loss, acc=acc)

    print(f"✅ Epoch {epoch+1} - Train Loss: {avg_loss:.4f} | Acc: {acc:.4f}")

    # === Validación
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
    with torch.no_grad():
        for inputs, labels in val_bar:
            inputs = inputs.to(device)
            labels = labels.to(device).float().unsqueeze(1)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

            avg_val_loss = val_loss / (val_bar.n + 1)
            val_acc = val_correct / val_total
            val_bar.set_postfix(loss=avg_val_loss, acc=val_acc)

    print(f"🔍 Epoch {epoch+1} - Val   Loss: {avg_val_loss:.4f} | Acc: {val_acc:.4f}")


mps
backbone.0.weight: requires_grad = False
backbone.1.weight: requires_grad = False
backbone.1.bias: requires_grad = False
backbone.4.0.conv1.weight: requires_grad = False
backbone.4.0.bn1.weight: requires_grad = False
backbone.4.0.bn1.bias: requires_grad = False
backbone.4.0.conv2.weight: requires_grad = False
backbone.4.0.bn2.weight: requires_grad = False
backbone.4.0.bn2.bias: requires_grad = False
backbone.4.0.conv3.weight: requires_grad = False
backbone.4.0.bn3.weight: requires_grad = False
backbone.4.0.bn3.bias: requires_grad = False
backbone.4.0.downsample.0.weight: requires_grad = False
backbone.4.0.downsample.1.weight: requires_grad = False
backbone.4.0.downsample.1.bias: requires_grad = False
backbone.4.1.conv1.weight: requires_grad = False
backbone.4.1.bn1.weight: requires_grad = False
backbone.4.1.bn1.bias: requires_grad = False
backbone.4.1.conv2.weight: requires_grad = False
backbone.4.1.bn2.weight: requires_grad = False
backbone.4.1.bn2.bias: requires_grad = False
back

Epoch 1/10 [Train]:   1%|          | 37/4073 [00:13<25:32,  2.63it/s, acc=0.709, loss=0.611]